In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [11]:
!move "C:\Users\jju53\OneDrive\바탕 화면\train_titanic.csv" .
!move "C:\Users\jju53\OneDrive\바탕 화면\test_titanic.csv" .

        1개 파일을 이동했습니다.


# 데이터 불러오기

* Survivied는 생존 여부(0은 사망, 1은 생존; train 데이터에서만 제공),
* Pclass는 사회경제적 지위(1에 가까울 수록 높음),
* SipSp는 배우자나 형제 자매 명 수의 총 합,
* Parch는 부모 자식 명 수의 총 합을 나타낸다.
* Embarked : 승선한 항구명(C : Cherbourg, Q : Queenstown, S : Southampton)
**Cherbourg : 셰르부르, 프랑스 서북부의 도시
**Queenstown : 퀸스타운 (뉴질랜드)
**Southampton : 영국 잉글랜드 남동부에 있는 항구도시


In [2]:
train=pd.read_csv('./train_titanic.csv')
train_df=pd.DataFrame(train).copy()
train_id=train['PassengerId'].tolist()



test=pd.read_csv('./test_titanic.csv')
test_df=pd.DataFrame(test).copy()
test_id=test_df['PassengerId'].tolist()


all_df=train_df.append(test_df)

all_df.reset_index(inplace=True)
all_df.tail(5)

,index,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
1304,413,1305,NaN,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S
1305,414,1306,NaN,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C
1306,415,1307,NaN,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S
1307,416,1308,NaN,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S
1308,417,1309,NaN,3,"Peter, Master. Michael J",male,NaN,1,1,2668,22.3583,NaN,C


In [3]:
all_df.isnull().sum()

index             0
PassengerId       0
Survived        418
Pclass            0
Name              0
Sex               0
Age             263
SibSp             0
Parch             0
Ticket            0
Fare              1
Cabin          1014
Embarked          2
dtype: int64

In [4]:
all_df['surname']=all_df.Name.str.split(',').str[1].str.split('.').str[0].str.replace(' ','')
all_df['surname'].value_counts()


Mr             757
Miss           260
Mrs            197
Master          61
Dr               8
Rev              8
Col              4
Mlle             2
Major            2
Ms               2
Sir              1
Dona             1
Capt             1
Don              1
Jonkheer         1
Mme              1
theCountess      1
Lady             1
Name: surname, dtype: int64

In [5]:
all_df[all_df['surname'] == 'Rev'] # 신부(남자)
all_df[all_df['surname'] == 'Capt']# 선장


# 영어 아닌 단어로 Miss를 뜻하고 있어 수정
all_df[all_df['surname'] == 'Mlle'] # Mlle=Miss
all_df[all_df['surname'] == 'Lady'] #Lady = Miss
all_df[all_df['surname'] == 'Mme']
#Mme=영부인=Mrs 그런데 실제 이 사람은 24세에 sipsp합이 0이라 miss로 바꿈
miss = all_df[all_df['surname'].str.contains('Mlle|Lady|Mme')].index.tolist()
for i in miss:
    all_df['surname'][i] = 'Miss'


# Mrs로 수정
all_df[all_df['surname'] == 'Dona']# 귀부인, Madam, lady
mrs = all_df[all_df['surname'].str.contains('Dona')].index.tolist()
for i in mrs:
    all_df['surname'][i] = 'Mrs'
    
# 귀족
all_df[all_df['surname'] == 'theCountess'] # 백작부인
all_df[all_df['surname'] == 'Don']#백작
all_df[all_df['surname'] == 'Sir'] # 영국에서 나이트 or 준남작
all_df[all_df['surname'] == 'Jonkheer'] # 네덜란드 작위없는 귀족
noblesse = all_df[all_df['surname'].str.contains('theCountess|Don|Sir|Jonkheer')].index.tolist()
for i in noblesse:
    all_df['surname'][i] = 'Noblesse'

# 군인
all_df[all_df['surname'] == 'Col'] # 대령(군인, 남자)
all_df[all_df['surname'] == 'Major'] # 소령
soldier = all_df[all_df['surname'].str.contains('Col|Major')].index.tolist()
for i in soldier:
    all_df['surname'][i] = 'Soldier'

    
    
    
all_df['surname'].value_counts()

,index,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,surname
149,149,150,0.0,2,"Byles, Rev. Thomas Roussel Davids",male,42.0,0,0,244310,13.000,NaN,S,Rev
150,150,151,0.0,2,"Bateman, Rev. Robert James",male,51.0,0,0,S.O.P. 1166,12.525,NaN,S,Rev
249,249,250,0.0,2,"Carter, Rev. Ernest Courtenay",male,54.0,1,0,244252,26.000,NaN,S,Rev
626,626,627,0.0,2,"Kirkland, Rev. Charles Leonard",male,57.0,0,0,219533,12.350,NaN,Q,Rev
848,848,849,0.0,2,"Harper, Rev. John",male,28.0,0,1,248727,33.000,NaN,S,Rev
886,886,887,0.0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.000,NaN,S,Rev
1040,149,1041,NaN,2,"Lahtinen, Rev. William",male,30.0,1,1,250651,26.000,NaN,S,Rev
1055,164,1056,NaN,2,"Peruschitz, Rev. Joseph Maria",male,41.0,0,0,237393,13.000,NaN,S,Rev


,index,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,surname
745,745,746,0.0,1,"Crosby, Capt. Edward Gifford",male,70.0,1,1,WE/P 5735,71.0,B22,S,Capt


,index,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,surname
641,641,642,1.0,1,"Sagesser, Mlle. Emma",female,24.0,0,0,PC 17477,69.3000,B35,C,Mlle
710,710,711,1.0,1,"Mayne, Mlle. Berthe Antonine (""Mrs de Villiers"")",female,24.0,0,0,PC 17482,49.5042,C90,C,Mlle


,index,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,surname
556,556,557,1.0,1,"Duff Gordon, Lady. (Lucille Christiana Sutherl...",female,48.0,1,0,11755,39.6,A16,C,Lady


,index,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,surname
369,369,370,1.0,1,"Aubart, Mme. Leontine Pauline",female,24.0,0,0,PC 17477,69.3,B35,C,Mme


...

C:\Users\jju53\anaconda3\lib\site-packages\ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,index,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,surname
647,647,648,1.0,1,"Simonius-Blumer, Col. Oberst Alfons",male,56.0,0,0,13213,35.500,A26,C,Col
694,694,695,0.0,1,"Weir, Col. John",male,60.0,0,0,113800,26.550,NaN,S,Col
1022,131,1023,NaN,1,"Gracie, Col. Archibald IV",male,53.0,0,0,113780,28.500,C51,C,Col
1093,202,1094,NaN,1,"Astor, Col. John Jacob",male,47.0,1,0,PC 17757,227.525,C62 C64,C,Col


,index,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,surname
449,449,450,1.0,1,"Peuchen, Major. Arthur Godfrey",male,52.0,0,0,113786,30.50,C104,S,Major
536,536,537,0.0,1,"Butt, Major. Archibald Willingham",male,45.0,0,0,113050,26.55,B38,S,Major


C:\Users\jju53\anaconda3\lib\site-packages\ipykernel_launcher.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Mr          757
Miss        264
Mrs         198
Master       61
Rev           8
Dr            8
Soldier       6
Noblesse      4
Ms            2
Capt          1
Name: surname, dtype: int64

# Missing data 채우기

## Age를 surname에 따라 평균으로 내서 대체해줌

In [6]:
all_df.isnull().sum()

index             0
PassengerId       0
Survived        418
Pclass            0
Name              0
Sex               0
Age             263
SibSp             0
Parch             0
Ticket            0
Fare              1
Cabin          1014
Embarked          2
surname           0
dtype: int64

In [7]:
age_miss= pd.DataFrame(all_df['Age'].groupby(all_df['surname']).mean(),)
age_miss.columns=['Age_mean']
age_miss
missing = all_df[all_df['Age'].isnull()].index.tolist()


for i in missing:
    if all_df['surname'][i] in age_miss.index.tolist():
        b = all_df['surname'][i]
        c=int(age_miss[age_miss.index == b]['Age_mean'])
        all_df['Age'][i]=c



,Age_mean
surname,
Capt,70.000000
Dr,43.571429
Master,5.482642
Miss,21.927991
Mr,32.252151
Mrs,37.005848
Ms,28.000000
Noblesse,40.000000
Rev,41.250000


C:\Users\jju53\anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


In [8]:
all_df.isnull().sum()

index             0
PassengerId       0
Survived        418
Pclass            0
Name              0
Sex               0
Age               0
SibSp             0
Parch             0
Ticket            0
Fare              1
Cabin          1014
Embarked          2
surname           0
dtype: int64

In [9]:
all_df.head()

,index,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,surname
0,0,1,0.0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,Mr
1,1,2,1.0,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,Mrs
2,2,3,1.0,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,Miss
3,3,4,1.0,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,Mrs
4,4,5,0.0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,Mr


## Fare missing
 * 큰 값이 있어서 median사용

In [10]:
all_df['Fare'].describe()

count    1308.000000
mean       33.295479
std        51.758668
min         0.000000
25%         7.895800
50%        14.454200
75%        31.275000
max       512.329200
Name: Fare, dtype: float64

In [11]:
#큰 값이 쏠려있어서 mean이 아닌 median으로 줌

all_df[all_df['Fare'].isnull()].index.tolist()
all_df['Fare'][1043]=14.4542
all_df.isnull().sum()

[1043]

C:\Users\jju53\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


index             0
PassengerId       0
Survived        418
Pclass            0
Name              0
Sex               0
Age               0
SibSp             0
Parch             0
Ticket            0
Fare              0
Cabin          1014
Embarked          2
surname           0
dtype: int64

## Embarked missing
 * 큰 값이 있어서 median사용

In [12]:
all_df['Embarked'].value_counts()

all_df[all_df['Embarked'].isnull()].index.tolist()
all_df['Embarked'][61]='S'
all_df['Embarked'][829]='S'

all_df.isnull().sum()

S    914
C    270
Q    123
Name: Embarked, dtype: int64

[61, 829]

C:\Users\jju53\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
C:\Users\jju53\anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


index             0
PassengerId       0
Survived        418
Pclass            0
Name              0
Sex               0
Age               0
SibSp             0
Parch             0
Ticket            0
Fare              0
Cabin          1014
Embarked          0
surname           0
dtype: int64

## Cabin missing

Fare, p class, surname, embarked

cabin의 알파벳이 앞쪽일수록 귀족,돈 많은 사람들일 경우가 많음

In [13]:
all_df['Fare_cat']=None

fare_cat=list(range(0,600,50))

fare_cat

for i in list(range(all_df['Fare'].count())):
    try:
        for a in list(range(len(fare_cat))):
            if fare_cat[a] <= all_df['Fare'][i] < fare_cat[a+1]:
                    all_df['Fare_cat'][i]=a
    except:
        break

all_df['Fare_cat'].value_counts()

[0, 50, 100, 150, 200, 250, 300, 350, 400, 450, 500, 550]

C:\Users\jju53\anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


0     1067
1      158
2       33
4       21
5       13
3       13
10       4
Name: Fare_cat, dtype: int64

In [14]:
all_df['Cabin_stair']=all_df['Cabin'].str[0]
all_df

,index,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,surname,Fare_cat,Cabin_stair
0,0,1,0.0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,Mr,0,NaN
1,1,2,1.0,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,Mrs,1,C
2,2,3,1.0,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,Miss,0,NaN
3,3,4,1.0,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,Mrs,1,C
4,4,5,0.0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,Mr,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1304,413,1305,NaN,3,"Spector, Mr. Woolf",male,32.0,0,0,A.5. 3236,8.0500,NaN,S,Mr,0,NaN
1305,414,1306,NaN,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C,Mrs,2,C
1306,415,1307,NaN,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S,Mr,0,NaN
1307,416,1308,NaN,3,"Ware, Mr. Frederick",male,32.0,0,0,359309,8.0500,NaN,S,Mr,0,NaN


In [15]:
a=pd.crosstab([all_df['Fare_cat'],all_df['Embarked'], all_df["Pclass"]], all_df['Cabin_stair'])
cabin_fillna=pd.DataFrame(a)
# 위의 값으로 모두 대체가능할 듯

In [16]:
a=pd.crosstab([all_df['Fare_cat'],all_df['Embarked'], all_df["Pclass"]], all_df['Cabin_stair'])
a

Cabin_stair                A   B   C   D   E   F  G  T
Fare_cat Embarked Pclass                              
0        C        1       10   4   8   2   0   0  0  0
                  2        0   0   0   4   0   0  0  0
                  3        0   0   0   0   0   3  0  0
         Q        2        0   0   0   0   1   0  0  0
                  3        0   0   0   0   0   1  0  0
         S        1        5   9  12  11  10   0  0  1
                  2        0   0   0   2   3  13  0  0
                  3        0   0   0   0   3   4  5  0
1        C        1        1  13  13  11   6   0  0  0
         Q        1        0   0   3   0   0   0  0  0
         S        1        6  17  18  13  13   0  0  0
2        C        1        0   3  12   3   5   0  0  0
         S        1        0   4   1   0   0   0  0  0
3        S        1        0   0   9   0   0   0  0  0
4        C        1        0   3   8   0   0   0  0  0
         S        1        0   3   4   0   0   0  0  0
5        C        1        0   6   0   0   0   0  0  0
         S        1        0   0   6   0   0   0  0  0
10       C        1        0   3   0   0   0   0  0  0

In [17]:

cabin_fillna=pd.DataFrame(a)
cabin_fillna_index = cabin_fillna.index.tolist() #(_, _, _)
cabin_fillna_index
cabin_fillna_col=cabin_fillna.columns.tolist()
cabin_fillna_col
cabin_fillna.values
#a=enumerate(cabin_fillna.values.max(axis=1))

b=(cabin_fillna.values == cabin_fillna.values.max(axis=1).reshape(-1,1)).tolist()

cabin_fillna_val=[]
for i in b:
    for index, value in enumerate(i):
            if value==True:
                cabin_fillna_v=index
                cabin_fillna_val.append(cabin_fillna_v)
                break



cabin_fillna_val


[(0, 'C', 1),
 (0, 'C', 2),
 (0, 'C', 3),
 (0, 'Q', 2),
 (0, 'Q', 3),
 (0, 'S', 1),
 (0, 'S', 2),
 (0, 'S', 3),
 (1, 'C', 1),
 (1, 'Q', 1),
 (1, 'S', 1),
 (2, 'C', 1),
 (2, 'S', 1),
 (3, 'S', 1),
 (4, 'C', 1),
 (4, 'S', 1),
 (5, 'C', 1),
 (5, 'S', 1),
 (10, 'C', 1)]

['A', 'B', 'C', 'D', 'E', 'F', 'G', 'T']

array([[10,  4,  8,  2,  0,  0,  0,  0],
       [ 0,  0,  0,  4,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  3,  0,  0],
       [ 0,  0,  0,  0,  1,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  1,  0,  0],
       [ 5,  9, 12, 11, 10,  0,  0,  1],
       [ 0,  0,  0,  2,  3, 13,  0,  0],
       [ 0,  0,  0,  0,  3,  4,  5,  0],
       [ 1, 13, 13, 11,  6,  0,  0,  0],
       [ 0,  0,  3,  0,  0,  0,  0,  0],
       [ 6, 17, 18, 13, 13,  0,  0,  0],
       [ 0,  3, 12,  3,  5,  0,  0,  0],
       [ 0,  4,  1,  0,  0,  0,  0,  0],
       [ 0,  0,  9,  0,  0,  0,  0,  0],
       [ 0,  3,  8,  0,  0,  0,  0,  0],
       [ 0,  3,  4,  0,  0,  0,  0,  0],
       [ 0,  6,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  6,  0,  0,  0,  0,  0],
       [ 0,  3,  0,  0,  0,  0,  0,  0]], dtype=int64)

[0, 3, 5, 4, 5, 2, 5, 6, 1, 2, 2, 2, 1, 2, 2, 2, 1, 2, 1]

In [18]:
missing_Cabin =all_df[all_df['Cabin'].isnull()].index.tolist()

            
for i in missing_Cabin:
    z=(all_df['Fare_cat'][i], all_df['Embarked'][i], all_df['Pclass'][i])            
    for x,y in enumerate(cabin_fillna_index):
        if y==z:           
            all_df['Cabin_stair'][i]=cabin_fillna_col[cabin_fillna_val[x]]
            break
            
all_df['Cabin_stair']

C:\Users\jju53\anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


0       G
1       C
2       G
3       C
4       G
       ..
1304    G
1305    C
1306    G
1307    G
1308    F
Name: Cabin_stair, Length: 1309, dtype: object

In [19]:
all_df['Cabin_stair'].isnull().sum()

31

* 위의 경우로 되지 않은 부분은 Pclass에 따라 대입

In [20]:
a=pd.crosstab([all_df["Pclass"]], all_df['Cabin_stair'])
a

all_df[all_df['Cabin'].isnull()]['Pclass'].value_counts()

Cabin_stair,A,B,C,D,E,F,G,T
Pclass,,,,,,,,
1,35,79,134,40,34,0,0,1
2,0,0,0,30,10,225,0,0
3,0,0,0,0,3,218,469,0


3    693
2    254
1     67
Name: Pclass, dtype: int64

In [21]:
f = all_df[all_df['Cabin'].isnull()].index.tolist()

for i in f:
    if all_df['Pclass'][i]==2:
        all_df['Cabin_stair'][i]='F'
    elif all_df['Pclass'][i]==3:
        all_df['Cabin_stair'][i]='G'



C:\Users\jju53\anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
C:\Users\jju53\anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [22]:

for i in list(range(0,1309)):
    if all_df['Sex'][i] == 'male':
        all_df['Sex'][i] = 1
    if all_df['Sex'][i] == 'female':
        all_df['Sex'][i] = 0
        
all_df=all_df.astype({'Sex':'int'})

C:\Users\jju53\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\jju53\anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [35]:
all_df

,index,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,surname,Fare_cat,Cabin_stair
0,0,1,0.0,3,"Braund, Mr. Owen Harris",1,22.0,1,0,A/5 21171,7.2500,NaN,S,Mr,0,G
1,1,2,1.0,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,38.0,1,0,PC 17599,71.2833,C85,C,Mrs,1,C
2,2,3,1.0,3,"Heikkinen, Miss. Laina",0,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,Miss,0,G
3,3,4,1.0,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,35.0,1,0,113803,53.1000,C123,S,Mrs,1,C
4,4,5,0.0,3,"Allen, Mr. William Henry",1,35.0,0,0,373450,8.0500,NaN,S,Mr,0,G
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1304,413,1305,NaN,3,"Spector, Mr. Woolf",1,32.0,0,0,A.5. 3236,8.0500,NaN,S,Mr,0,G
1305,414,1306,NaN,1,"Oliva y Ocana, Dona. Fermina",0,39.0,0,0,PC 17758,108.9000,C105,C,Mrs,2,C
1306,415,1307,NaN,3,"Saether, Mr. Simon Sivertsen",1,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S,Mr,0,G
1307,416,1308,NaN,3,"Ware, Mr. Frederick",1,32.0,0,0,359309,8.0500,NaN,S,Mr,0,G


In [37]:
all_df['Embarked'].unique()
all_df['surname'].unique()
all_df['Cabin_stair'].unique()

array(['S', 'C', 'Q'], dtype=object)

array(['Mr', 'Mrs', 'Miss', 'Master', 'Noblesse', 'Rev', 'Dr', 'Ms',
       'Soldier', 'Capt'], dtype=object)

array(['G', 'C', 'E', 'F', 'D', 'A', 'B', 'T'], dtype=object)

In [40]:
all_df_list=[all_df]
for dataset in all_df_list:
    dataset.loc[dataset['Embarked']=='S', 'Embarked']=0,
    dataset.loc[dataset['Embarked']=='C', 'Embarked']=1,
    dataset.loc[dataset['Embarked']=='Q', 'Embarked']=2

    
for dataset in all_df_list:
    dataset.loc[dataset['surname']=='Mr', 'surname']=0,
    dataset.loc[dataset['surname']=='Mrs', 'surname']=1,
    dataset.loc[dataset['surname']=='Miss', 'surname']=2,
    dataset.loc[dataset['surname']=='Master', 'surname']=3,
    dataset.loc[dataset['surname']=='Noblesse', 'surname']=4,
    dataset.loc[dataset['surname']=='Rev', 'surname']=5,
    dataset.loc[dataset['surname']=='Dr', 'surname']=6,
    dataset.loc[dataset['surname']=='Ms', 'surname']=7,
    dataset.loc[dataset['surname']=='Soldier', 'surname']=8,
    dataset.loc[dataset['surname']=='Capt', 'surname']=9
    
for dataset in all_df_list:
    dataset.loc[dataset['Cabin_stair']=='G', 'Cabin_stair']=0,
    dataset.loc[dataset['Cabin_stair']=='C', 'Cabin_stair']=1,
    dataset.loc[dataset['Cabin_stair']=='E', 'Cabin_stair']=2,
    dataset.loc[dataset['Cabin_stair']=='F', 'Cabin_stair']=3,
    dataset.loc[dataset['Cabin_stair']=='D', 'Cabin_stair']=4,
    dataset.loc[dataset['Cabin_stair']=='A', 'Cabin_stair']=5,
    dataset.loc[dataset['Cabin_stair']=='B', 'Cabin_stair']=6,
    dataset.loc[dataset['Cabin_stair']=='T', 'Cabin_stair']=7

In [41]:
all_df.head()

,index,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,surname,Fare_cat,Cabin_stair
0,0,1,0.0,3,"Braund, Mr. Owen Harris",1,22.0,1,0,A/5 21171,7.2500,NaN,0,0,0,0
1,1,2,1.0,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,38.0,1,0,PC 17599,71.2833,C85,1,1,1,1
2,2,3,1.0,3,"Heikkinen, Miss. Laina",0,26.0,0,0,STON/O2. 3101282,7.9250,NaN,0,2,0,0
3,3,4,1.0,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,35.0,1,0,113803,53.1000,C123,0,1,1,1
4,4,5,0.0,3,"Allen, Mr. William Henry",1,35.0,0,0,373450,8.0500,NaN,0,0,0,0


# Model fitting_Random forest

In [42]:
# train data의 y값

all=all_df.copy()

train_y=pd.DataFrame()
for i in list(range(0,1309)):
    if all['PassengerId'][i] in train_id:
        a=all[all['PassengerId']==i+1]
        train_y=train_y.append(a)
        

train_y=train_y['Survived']
train_y=train_y.values
len(train_y)

all_x=all.drop(['Survived'],axis=1).copy()



891

In [43]:
all_x.drop(['index','Name','Ticket','Cabin'], axis=1, inplace=True)

In [44]:
all_x.head(1)

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,surname,Fare_cat,Cabin_stair
0,1,3,1,22.0,1,0,7.25,0,0,0,0


In [45]:
train_x=pd.DataFrame()
test_x=pd.DataFrame()

for i in list(range(0,1309)):
    if all_x['PassengerId'][i] in train_id:
        a=all_x[all_x['PassengerId'] == i+1]
        train_x=train_x.append(a)
        
    elif all_x['PassengerId'][i] in test_id:
        b=all_x[all_x['PassengerId'] == i+1]
        test_x=test_x.append(b)



In [46]:
train_x=train_x.drop(['PassengerId'],axis=1)
test_x=test_x.drop(['PassengerId'],axis=1)




In [47]:
train_x=train_x.values
test_x=test_x.values


In [50]:
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics 

forest = RandomForestClassifier(n_estimators=100)
forest.fit(train_x, train_y)

train_y_pred = forest.predict(train_x).astype(int)


len(test_y_pred)
len(test_id)

print('정확도 :', metrics.accuracy_score(train_y, train_y_pred))

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

418

418

정확도 : 0.9842873176206509


In [51]:
test_y_pred = forest.predict(test_x).astype(int)
len(test_y_pred)
len(test_id)

submission=pd.DataFrame({"PassengerId":test_id,"Survived":test_y_pred})
submission.to_csv('submission_200619_RamdomForest.csv', index=False)

submission.head()

418

418

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,1
4,896,0
